In [1]:
"""dython 설치를 0.5.1로 해야 함"""

%reload_ext autoreload
%autoreload 2 
from model.ctabgan import CTABGAN
from model.eval.evaluation import get_utility_metrics,stat_sim,privacy_metrics
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import importlib

c:\Users\RohSeungchan\anaconda3\envs\torch-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\RohSeungchan\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
c:\Users\RohSeungchan\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\RohSeungchan\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
num_exp = 1
dataset = "cabs"
real_path = "CHECK_DATASETS/train_hearts_data.csv"
fake_file_root = "Fake_Datasets"

In [3]:
# df = pd.read_csv(real_path)
# df.head(2)

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,HeartDisease
0,29.21,No,No,No,0.0,0.0,No,Male,55-59,White,No,Yes,Excellent,8.0,No,No,No,No
1,25.84,No,No,No,4.0,0.0,No,Male,80 or older,White,No,Yes,Excellent,6.0,No,No,Yes,No


In [4]:
# continuous_list = []
# categorical_list = []
# for col in df.columns:
#     if df[col].dtypes != 'O':
#         continuous_list.append(col)
#     else:
#         categorical_list.append(col)

# continuous_list

['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']

In [8]:
"""종속변수는 categorical columns와 problem_type에만 넣어야 함"""
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20, # test_ratio랑 상관없이 입력 데이터 크기만큼 학습하고 생성함
                 categorical_columns = ['HeartDisease','Smoking','AlcoholDrinking','Stroke','DiffWalking','Sex','AgeCategory','Race','Diabetic','PhysicalActivity','GenHealth','Asthma','KidneyDisease','SkinCancer'], 
                 log_columns = [],
                 mixed_columns= {'PhysicalHealth':[30.0],'MentalHealth':[30.0]},
                 integer_columns = ['PhysicalHealth','MentalHealth','SleepTime'],
                 problem_type= {"Classification": 'HeartDisease'},
                 epochs= 100) 

for i in range(2):
    synthesizer.fit()
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+"CTABGAN_fake_Hearts_{exp}_230512.csv".format(exp=i), index= False)

100%|██████████| 100/100 [51:56<00:00, 31.16s/it]


Finished training in 3194.9413800239563  seconds.


100%|██████████| 100/100 [52:20<00:00, 31.41s/it]


Finished training in 3219.667867422104  seconds.


In [9]:
fake_paths = glob.glob(fake_file_root+"/"+dataset+"/"+"*")
fake_paths

[]

In [ ]:
classifiers_list = ["lr","dt","rf","mlp"]
result_mat = get_utility_metrics(real_path,fake_paths,"MinMax",classifiers_list, test_ratio = 0.20)

result_df  = pd.DataFrame(result_mat,columns=["Acc","AUC","F1_Score"])
result_df.index = classifiers_list
result_df

In [ ]:
adult_categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
stat_res_avg = []
for fake_path in fake_paths:
    stat_res = stat_sim(real_path,fake_path,adult_categorical)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg).mean(axis=0).reshape(1,3),columns=stat_columns)
stat_results

In [ ]:
priv_res_avg = []
for fake_path in fake_paths:
    priv_res = privacy_metrics(real_path,fake_path)
    priv_res_avg.append(priv_res)
    
privacy_columns = ["DCR between Real and Fake (5th perc)","DCR within Real(5th perc)","DCR within Fake (5th perc)","NNDR between Real and Fake (5th perc)","NNDR within Real (5th perc)","NNDR within Fake (5th perc)"]
privacy_results = pd.DataFrame(np.array(priv_res_avg).mean(axis=0).reshape(1,6),columns=privacy_columns)
privacy_results